In [1]:
!cp -r /kaggle/input/col774a3/* /kaggle/working/

In [2]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

# Set default thresholds for each class
class_thresholds = {i: 0.9 for i in range(100)}  # Default threshold for each class

# Custom function to calculate the score
def calculate_score(true_labels: pd.Series, predicted_labels: pd.Series, accuracy_threshold: float = 0.7, gamma: float = 5.0) -> float:
    data = pd.DataFrame({'True_label': true_labels, 'Predicted_label': predicted_labels})
    filtered_df = data[data['Predicted_label'] != -1]
    all_classes = list(range(100))
    
    sum_of_correctly_classified_high_accuracy = 0
    sum_of_correctly_classified_low_accuracy = 0
    accuracy_per_class = {}
    
    grouped = filtered_df.groupby('Predicted_label')
    for name, group in grouped:
        accuracy = (group['True_label'] == group['Predicted_label']).sum() / len(group)
        accuracy_per_class[name] = accuracy
    
    for cls in all_classes:
        total = len(filtered_df[filtered_df['Predicted_label'] == cls])
        class_accuracy = accuracy_per_class.get(cls, 0.0)
        if class_accuracy >= accuracy_threshold:
            sum_of_correctly_classified_high_accuracy += total
        else:
            sum_of_correctly_classified_low_accuracy += total

    final_score = sum_of_correctly_classified_high_accuracy - gamma * sum_of_correctly_classified_low_accuracy
    return round(final_score / 2)

# Custom function to apply thresholds and return predictions
def apply_class_thresholds(probabilities, class_thresholds):
    predicted_labels = []
    for i in range(probabilities.size(0)):
        max_prob, predicted_class = torch.max(probabilities[i], 0)
        if max_prob >= class_thresholds[predicted_class.item()]:
            predicted_labels.append(predicted_class.item())
        else:
            predicted_labels.append(-1)
    return torch.tensor(predicted_labels)

# CIFAR100Dataset for loading data from pkl files
class CIFAR100Dataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return image, label

# Test function that loads from test.pkl and applies threshold-based predictions
def test_with_thresholds_from_pkl(epoch, model, test_pkl_file, device, class_thresholds):
    test_dataset = CIFAR100Dataset(test_pkl_file)  # Load test.pkl
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    model.eval()
    correct = 0
    total = 0
    skipped_predictions = 0
    
    true_labels = []
    predicted_labels = []
    predictions_for_csv = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)

            # Apply class-specific thresholds
            predictions = apply_class_thresholds(probabilities, class_thresholds).to(device)

            total += targets.size(0)
            correct_predictions = predictions.eq(targets).sum().item()
            skipped_predictions += (predictions == -1).sum().item()

            correct += correct_predictions
            
            true_labels.extend(targets.cpu().numpy())
            predicted_labels.extend(predictions.cpu().numpy())

            # Save predictions for CSV
            predictions_for_csv.extend(predictions.cpu().numpy())

    # Save predictions and true labels to CSV
    prediction_df = pd.DataFrame({
        'ID': list(range(len(predictions_for_csv))),
        'Predicted_label': predictions_for_csv,
        'True_label': true_labels
    })
    prediction_df.to_csv(f'predictions_epoch_{epoch}.csv', index=False)
    print(f'Predictions for epoch {epoch} saved to CSV.')

    # Calculate the test accuracy
    test_accuracy = 100. * correct / total
    print(f'Test set: Accuracy: {correct}/{total} ({test_accuracy:.2f}%) | Skipped: {skipped_predictions}')

    return correct, total

# WideResNeXt Block (same as before)
class WideResNeXtBlock(nn.Module):
    expansion = 2
    def __init__(self, in_planes, planes, stride=1, cardinality=32, widen_factor=2):
        super(WideResNeXtBlock, self).__init__()
        D = cardinality * widen_factor
        self.conv1 = nn.Conv2d(in_planes, D, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(D)
        self.conv2 = nn.Conv2d(D, D, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(D)
        self.conv3 = nn.Conv2d(D, planes * WideResNeXtBlock.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * WideResNeXtBlock.expansion)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * WideResNeXtBlock.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * WideResNeXtBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * WideResNeXtBlock.expansion)
            )
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# WideResNeXt Model (same as before)
class WideResNeXt(nn.Module):
    def __init__(self, block, num_blocks, cardinality=32, widen_factor=2, num_classes=100):
        super(WideResNeXt, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, cardinality=cardinality, widen_factor=widen_factor)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, cardinality=cardinality, widen_factor=widen_factor)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, cardinality=cardinality, widen_factor=widen_factor)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, cardinality=cardinality, widen_factor=widen_factor)
        self.linear = nn.Linear(512 * WideResNeXtBlock.expansion, num_classes)
    
    def _make_layer(self, block, planes, num_blocks, stride, cardinality, widen_factor):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, cardinality, widen_factor))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# WideResNeXt-101 Model Definition
def WideResNeXt101():
    return WideResNeXt(WideResNeXtBlock, [3, 4, 23, 3], cardinality=32, widen_factor=2)

# Load CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

test_pkl_file = 'test.pkl'  # Path to your test.pkl file

# Model, loss, optimizer, and scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = WideResNeXt101().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop
for epoch in range(0, 100):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch}: Loss: {running_loss / (batch_idx + 1)}')
    
    # Test model with thresholds after each epoch using test.pkl
    correct, total = test_with_thresholds_from_pkl(epoch, model, test_pkl_file, device, class_thresholds)
    
    scheduler.step()


100%|██████████| 169001437/169001437 [00:02<00:00, 58932004.56it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Epoch 0: Loss: 4.509624347662377


/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Predictions for epoch 0 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 20000
Epoch 1: Loss: 3.765388569868434
Predictions for epoch 1 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 19960
Epoch 2: Loss: 3.334648060981575
Predictions for epoch 2 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 19901
Epoch 3: Loss: 2.9378749690092434
Predictions for epoch 3 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 19748
Epoch 4: Loss: 2.5560292014685433
Predictions for epoch 4 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 18959
Epoch 5: Loss: 2.260148223708658
Predictions for epoch 5 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 18498
Epoch 6: Loss: 2.051713596829368
Predictions for epoch 6 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 18259
Epoch 7: Loss: 1.9029012674565815
Predictions for epoch 7 saved to CSV.
Test set: Accuracy: 0/20000 (0.00%) | Skipped: 17238
Epoch 8: Loss: 1.7950184387928994
Pred

In [3]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# import pandas as pd
# from torch.optim.lr_scheduler import StepLR
# import os

# # Set default thresholds for each class
# class_thresholds = {i: 0.9 for i in range(100)}  # Default threshold for each class

# # Custom function to calculate the score
# def calculate_score(true_labels: pd.Series, predicted_labels: pd.Series, accuracy_threshold: float = 0.7, gamma: float = 5.0) -> float:
#     data = pd.DataFrame({'True_label': true_labels, 'Predicted_label': predicted_labels})
#     filtered_df = data[data['Predicted_label'] != -1]
#     all_classes = list(range(100))
    
#     sum_of_correctly_classified_high_accuracy = 0
#     sum_of_correctly_classified_low_accuracy = 0
#     accuracy_per_class = {}
    
#     grouped = filtered_df.groupby('Predicted_label')
#     for name, group in grouped:
#         accuracy = (group['True_label'] == group['Predicted_label']).sum() / len(group)
#         accuracy_per_class[name] = accuracy
    
#     for cls in all_classes:
#         total = len(filtered_df[filtered_df['Predicted_label'] == cls])
#         class_accuracy = accuracy_per_class.get(cls, 0.0)
#         if class_accuracy >= accuracy_threshold:
#             sum_of_correctly_classified_high_accuracy += total
#         else:
#             sum_of_correctly_classified_low_accuracy += total

#     final_score = sum_of_correctly_classified_high_accuracy - gamma * sum_of_correctly_classified_low_accuracy
#     return round(final_score / 2)

# # Custom function to apply thresholds and return predictions
# def apply_class_thresholds(probabilities):
#     predicted_labels = []
#     for i in range(probabilities.size(0)):
#         max_prob, predicted_class = torch.max(probabilities[i], 0)
#         if max_prob >= class_thresholds[predicted_class.item()]:
#             predicted_labels.append(predicted_class.item())
#         else:
#             predicted_labels.append(-1)
#     return torch.tensor(predicted_labels)

# # Test with class-specific thresholds and generate CSV
# def test_with_thresholds(epoch, true_labels, predicted_labels):
#     global best_accuracy
#     model.eval()
#     test_loss = 0
#     correct = 0
#     wrong = 0
#     total = 0
#     skipped_predictions = 0
    
#     predictions_for_csv = []

#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(testloader):
#             inputs, targets = inputs.to(device), targets.to(device)
#             outputs = model(inputs)
#             probabilities = F.softmax(outputs, dim=1)
            
#             # Apply class-specific thresholds and move predictions to the same device as targets
#             predictions = apply_class_thresholds(probabilities).to(device)
            
#             total += targets.size(0)
#             correct_predictions = predictions.eq(targets).sum().item()
#             wrong_predictions = (predictions != -1).sum().item() - correct_predictions
#             skipped_predictions += (predictions == -1).sum().item()

#             correct += correct_predictions
#             wrong += wrong_predictions
            
#             true_labels.extend(targets.cpu().numpy())
#             predicted_labels.extend(predictions.cpu().numpy())

#             # Save predictions for CSV
#             predictions_for_csv.extend(predictions.cpu().numpy())

#     # Save predictions to CSV
#     prediction_df = pd.DataFrame({'ID': list(range(len(predictions_for_csv))), 'Predicted_label': predictions_for_csv})
#     prediction_df.to_csv(f'predictions_epoch_{epoch}.csv', index=False)
#     print(f'Predictions for epoch {epoch} saved to CSV.')

#     # Calculate the test accuracy
#     test_accuracy = 100. * correct / total
#     print(f'Test set: Accuracy: {correct}/{total} ({test_accuracy:.2f}%) | Skipped: {skipped_predictions}')
    
#     return correct, wrong, total

# # Load CIFAR-100 dataset
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
# ])

# trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
# testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# # Model, loss, optimizer, and scheduler
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = WideResNeXt101().to(device)

# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
# scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# # Example Training Loop
# true_labels = []
# predicted_labels = []

# for epoch in range(0, 100):
#     model.train()
#     running_loss = 0.0
#     for batch_idx, (inputs, targets) in enumerate(trainloader):
#         inputs, targets = inputs.to(device), targets.to(device)
        
#         # Zero the parameter gradients
#         optimizer.zero_grad()
        
#         # Forward + Backward + Optimize
#         outputs = model(inputs)
#         loss = F.cross_entropy(outputs, targets)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
    
#     print(f'Epoch {epoch}: Loss: {running_loss / (batch_idx + 1)}')
    
#     # Test model with thresholds after each epoch and generate predictions CSV
#     correct, wrong, total = test_with_thresholds(epoch, true_labels, predicted_labels)
    
#     # Calculate and print the score
#     final_score = calculate_score(pd.Series(true_labels), pd.Series(predicted_labels))
#     print(f"Epoch {epoch}: Correct predictions: {correct}, Wrong predictions: {wrong}, Total: {total}, Score: {final_score}")
    
#     scheduler.step()
